In [65]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time
import sys
sys.path.append('../../GlobalRepo/')
from global_repo import *


client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [66]:
program_id = AVATAR_PROGRAM_ID
# program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
# program_id_account_meta

In [67]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [68]:
def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [69]:
payer_keypair = Keypair.from_secret_key(base58.b58decode("mH4PkhAttpFXSrntbpiJgC8J6wHYFDK5PFWcPN53hMJKziWQucux7MFiW7uXnqKsAY3XBnxUtyVKnomZnD9JxHW"))
client.request_airdrop(payer_keypair.public_key, 2*10**9)

{'jsonrpc': '2.0',
 'result': '2Ru2TPbuinEvWFYttdwXAMmx9n4LGU6SibmhFPVmQWseGgLLL1mtP5NnQHKwo7P6CJYJscEuZqzhQUDwNVecLYg9',
 'id': 1}

In [70]:
payer_keypair.public_key

2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp

In [71]:
dsol_mint = DSOL_MINT_ID

In [72]:
seed = bytes([25,215,212,154,213,198,45,123]*4)
vault_keypair = Keypair.from_seed(seed)
vault = VAULT
# client.request_airdrop(vault_keypair.public_key, 2*10**9)
# time.sleep(5)

vault_dsol_token_account = instructions.get_associated_token_address(vault, dsol_mint)
vault_keypair.public_key

3HqXocfan3tVv3N4tTAFhyVePkP856jxngQ6pQxdupXP

In [73]:
mint_account = Keypair.generate()

payer_dsol_token_account = instructions.get_associated_token_address(payer_keypair.public_key, dsol_mint)
payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(vault, True, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)


In [74]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b"avatar_mint_authority_pda"], program_id)
(avatar_data_pda, avatar_pda_nonce) = PublicKey.find_program_address([b'avatar_data_pda', bytes(mint_account_meta.pubkey)], program_id)
(sales_pda, sales_nonce) = PublicKey.find_program_address([b'dsol_avatar_sales_pda'], program_id)
metadata_program_pubkey = METAPLEX_PROGRAM_ID
(metadata_pda, metadata_nonce) = PublicKey.find_program_address([b'metadata', bytes(metadata_program_pubkey), bytes(mint_account_meta.pubkey)], metadata_program_pubkey)


avatar_data_pda_meta = AccountMeta(avatar_data_pda, False, True)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
sales_pda_meta = AccountMeta(sales_pda, False, True)
mint_authority_meta = AccountMeta(mint_authority, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)


dsol_mint_meta = AccountMeta(dsol_mint, False, False)
vault_dsol_token_account_meta = AccountMeta(vault_dsol_token_account, False, True)
payer_dsol_token_account_meta = AccountMeta(payer_dsol_token_account, False, True)

print(dsol_mint)
print(vault_keypair.public_key)
print(vault_dsol_token_account)
print(payer_keypair.public_key)
print(payer_dsol_token_account)
print(mint_authority)
print(spl_program_meta)
print(rent_account_meta)
print(associated_token_address)
print(sales_pda)

HSYdApQF2FKVNnX6FQ1ySYstGLe5Gp8NseHVFavWye66
3HqXocfan3tVv3N4tTAFhyVePkP856jxngQ6pQxdupXP
H5bXtRguFwmif1VLQAVcYZwA5UwBQ8qUMders9TXLiQ7
2ZHc9QxDDaJwqNEFzpAGUrvxCWZSNnXSffHxV9hG2axp
fWNUKgUNPKrHkLDvUDq379bFmE6m7MR1MGgiBSpPPWC
CaJrkP1duqCUtXqEoYiwT2hNRYVyziU7jwUHPkstkE4K
AccountMeta(pubkey=TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA, is_signer=False, is_writable=False)
AccountMeta(pubkey=SysvarRent111111111111111111111111111111111, is_signer=False, is_writable=False)
8xk4TAWyS4G3GPLaxmAExMBfJYVCozcsqCLmrASrEAgV
3TGD8CqyMPjz9ruuhU1dZKjkHVjHBqr1xrsMYQq5EFbZ


In [75]:

# def init_vault():
#     accounts = [
#         payer_account_meta,
#         vault_account_meta,
#         vault_dsol_token_account_meta,
#         dsol_mint_meta,
#         sys_program_meta,
#         spl_program_meta,
#         rent_account_meta,

#         associated_program_meta,
#         # sys_program_meta,
#         # sys_program_meta,
#     ]

#     transaction = Transaction()
#     transaction.add(
#         TransactionInstruction(
#             accounts,
#             program_id,
#             bytes([9])
#         )
#     )
    
#     return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, vault_keypair)['result']}?cluster=devnet"

# init_vault()

# # time.sleep(5)
mint_account_meta.pubkey

Bh1DsFGZT5bNNjGvbv9VPKCRkQYXEyHxsyYkKiDPU42k

In [76]:
program_id

ASV3UahwSK3oRmNkkbyhFNSRSaccL2jJvAYCYQa8RSgG

In [77]:
def mint_avatar():
    accounts = [
        # program_id_account_meta,
        payer_account_meta,
        payer_dsol_token_account_meta,
        AccountMeta(vault,False, True), 
        vault_dsol_token_account_meta,
        mint_account_meta, 
        rent_account_meta, 
        associated_account_meta,
        spl_program_meta,
        AccountMeta(SYS_PROGRAM_ID, False, False),
        mint_authority_meta,
        avatar_data_pda_meta,
        sales_pda_meta,
        AccountMeta(metadata_pda, False, True),
        sysvar_clock_meta,

        spl_program_meta,
        AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
        spl_program_meta,
        spl_program_meta,
        AccountMeta(metadata_program_pubkey, False, False),
    ]   

    transaction = Transaction()
    transaction.add(TransactionInstruction(
        accounts,
        program_id,
        bytes([0])
    ))
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

mint_avatar()

'https://explorer.solana.com/tx/3ryR8hCtfWXYpFTuUNeKNsBexgRg6YRhrTdNNDh4by8QfvAi3ZT3MtvmegU8tWtoLHsQ6avGWdSwkEJ6QGJS954Q?cluster=devnet'

In [78]:
StructId = Enum(
    "AvatarSales0_0_1",
    "AvatarData0_0_1",
    "RentContainerData0_0_1",
    "AccountRentSpace0_0_1",
    "CollectionData0_0_1",
    "ContainerData0_0_1",
    "GovernorData0_0_1",
    "GovernorSales0_0_1",
    "Proposals0_0_1",
    "GovernorVote0_0_1",
    "GovernorsVote0_0_1",
    "ProposalResult0_0_1",
    "Proposal0_0_1",
    "TableSales0_0_1",
    "TableData0_0_1",
    "ProposalNumGovernors0_0_1",
    "ProposalLockGovernor0_0_1",
    "DsolData0_0_1",
    enum_name="StructId"
)

accountSchema = CStruct(
    "struct_id"/ U32,
    "vault_total" / F32,
    "counter" / U32,
    "rent_min_listed" / U32,
    "rent_max_listed" / U32,
    "rent_max_ever" / U32
)

getAccountInfo(client,sales_pda)

Container(struct_id=0, vault_total=123.6152114868164, counter=52, rent_min_listed=0, rent_max_listed=0, rent_max_ever=0)

In [79]:



accountSchema = CStruct(
    "struct_id" / U8,
    "date_created" / U32,
    "unlockable_date" / U32,
    "numeration" /U32,
    "rarity" / U8,
    "level" / U8,
    "xp" / U32,
    "rent_state" / U8,
    "use_authority" / U8[32],
    "rent_bump" / U32,
)


(locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

getAccountInfo(client,PublicKey("4tPJ2U9yRfKejCkPwHTgWCYjV728UmM5X4nrM5A1zHQX"))

PDA: (GWwMrQCJWdaL1kahndZ5VaAWtdSJkUu77uGM2VZfEiFh, 254)


Container(struct_id=1, date_created=1645973032, unlockable_date=1645973032, numeration=4, rarity=2, level=0, xp=0, rent_state=0, use_authority=ListContainer([23, 33, 12, 153, 230, 93, 161, 21, 203, 134, 227, 249, 98, 231, 6, 246, 65, 15, 155, 136, 121, 241, 203, 109, 115, 234, 247, 41, 223, 105, 164, 97]), rent_bump=0)

In [80]:
# time.sleep(120)

# new_accounts = [
#     mint_account_meta,
#     associated_account_meta,
#     spl_program_meta,
#     mint_authority_meta,
#     avatar_data_pda_meta,
#     sysvar_clock_meta,

#     spl_program_meta
# ]

# transaction = Transaction()
# transaction.add(TransactionInstruction(
#     new_accounts,
#     program_id,
#     bytes([1])
# ))
# f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"